In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
from pyrosm import OSM, get_data
import geopandas as gpd
import pandas as pd
from sklearn.neighbors import BallTree
import numpy as np
import mapclassify as mc
# import matplotlib.pyplot as plt
import time
import networkx as nx
import igraph as ig
import osmnx as ox
import folium
import json
from datetime import datetime, timedelta
from folium import PolyLine, Marker
from folium.plugins import BeautifyIcon
from math import radians, cos, sin, asin, sqrt
import random as r
import math


In [3]:
import time
tiempo_inicio = time.time()
# Coordenadas para Santiago Centro
lat, lon = -33.456574386393314, -70.65970917320084

# Ajusta los límites usando el valor de delta
delta = 0.09  # Menor valor para un área más pequeña
north = lat + delta
south = lat - delta
east = lon + delta
west = lon - delta

# Obtiene el grafo usando los límites ajustados
G = ox.graph_from_bbox(north, south, east, west, network_type='drive')
G = ox.speed.add_edge_speeds(G)
G = ox.speed.add_edge_travel_times(G)

almacen = {"id": "0", "lat": "-33.456574386393314", "long": "-70.65970917320084", "demand": 0}

# Datos de los clientes
with open("nodos.json", 'r', encoding='utf-8') as archivo:
    nodos = json.load(archivo)


nodos.insert(0, almacen)  # Asegúrate de que el almacén es el primer nodo en la lista


In [4]:
parametros = {}
parametros['cant_nodos'] = 31
parametros['cant_hormigas'] = 30
parametros['cant_iteraciones'] = 100
parametros['limite_tiempo'] = 1000
parametros['feromona_inicial'] = r.random()
parametros['alfa'] = 0.669
parametros['beta'] = 4.626
parametros['K'] = 0.413
parametros['Q'] = 0.498
parametros['evaporacion'] = 0.016
parametros['semilla'] = 1020
parametros['costo_dron'] = 0.33
parametros['costo_camion'] = 3.3
parametros['velocidad_camion'] = 55
parametros['velocidad_dron'] = 35
parametros['max_vuelo'] = 3000
parametros['tiempo_espera_camion'] = 6 * 60
parametros['tiempo_espera_dron'] = 2 * 60
parametros['tiempo_recuperacion_dron'] = 1 * 60
parametros['tiempo_servicio_camion'] = 1 * 60
parametros['tiempo_servicio_dron'] = 1 * 60

In [5]:
def distancia_camion(nodo1, nodo2, G):
    source_node = ox.distance.nearest_nodes(G, X=float(nodo1['long']), Y=float(nodo1['lat']))
    target_node = ox.distance.nearest_nodes(G, X=float(nodo2['long']), Y=float(nodo2['lat']))
    ruta = ox.shortest_path(G, source_node, target_node, weight='length')
    return sum(ox.utils_graph.get_route_edge_attributes(G, ruta, 'length'))

def haversine(lat1, lon1, lat2, lon2):
    # Convertir coordenadas de grados a radianes
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Fórmula de Haversine
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # Radio de la Tierra en kilómetros
    return c * r

def distancia_dron(nodo1, nodo2):
    return haversine(float(nodo1['lat']), float(nodo1['long']), float(nodo2['lat']), float(nodo2['long']))

def matriz_distancias_camion(lista_nodos, G):
    cant_nodos = len(lista_nodos)
    matriz_distancias_camion = [[0 for _ in range(cant_nodos)] for _ in range(cant_nodos)]
    for i in range(cant_nodos):
        for j in range(i + 1, cant_nodos):
            distancia = distancia_camion(lista_nodos[i], lista_nodos[j], G)
            matriz_distancias_camion[i][j] = matriz_distancias_camion[j][i] = distancia/1000
    return matriz_distancias_camion

def matriz_distancias_dron(lista_nodos):
    cant_nodos = len(lista_nodos)
    matriz_distancias_dron = [[0 for _ in range(cant_nodos)] for _ in range(cant_nodos)]
    for i in range(cant_nodos):
        for j in range(i + 1, cant_nodos):
            distancia = distancia_dron(lista_nodos[i], lista_nodos[j])
            matriz_distancias_dron[i][j] = matriz_distancias_dron[j][i] = distancia
    return matriz_distancias_dron

def calcular_matrices_costos(matriz_dist_camion, costo_camion, matriz_dist_dron, costo_dron):
    cant_nodos = len(matriz_dist_camion)  # Asumiendo que ambas matrices tienen el mismo tamaño
    matriz_costos_camion = [[0 for _ in range(cant_nodos)] for _ in range(cant_nodos)]
    matriz_costos_dron = [[0 for _ in range(cant_nodos)] for _ in range(cant_nodos)]

    for i in range(cant_nodos):
        for j in range(cant_nodos):
            matriz_costos_camion[i][j] = matriz_dist_camion[i][j] * costo_camion
            matriz_costos_dron[i][j] = matriz_dist_dron[i][j] * costo_dron

    return matriz_costos_camion, matriz_costos_dron




def calcular_matrices_tiempos(matriz_dist_camion, velocidad_camion, matriz_dist_dron, velocidad_dron):
    cant_nodos = len(matriz_dist_camion)
    matriz_tiempos_camion = [[0 for _ in range(cant_nodos)] for _ in range(cant_nodos)]
    matriz_tiempos_dron = [[0 for _ in range(cant_nodos)] for _ in range(cant_nodos)]

    for i in range(cant_nodos):
        for j in range(cant_nodos):
            if matriz_dist_camion[i][j] != 0:
                matriz_tiempos_camion[i][j] = (matriz_dist_camion[i][j] / velocidad_camion) * 3600
            if matriz_dist_dron[i][j] != 0:
                matriz_tiempos_dron[i][j] = (matriz_dist_dron[i][j] / velocidad_dron) * 3600

    return matriz_tiempos_camion, matriz_tiempos_dron

def matriz_feromonas(lista_nodos,f0):
    cant_nodos = len(lista_nodos)
    matriz_feromonas = []
    for i in range(cant_nodos):
        matriz_feromonas.append([0 if i == j else f0 for j in range(cant_nodos)])

    return matriz_feromonas


In [6]:

matriz_dist_camion = matriz_distancias_camion(nodos, G)
matriz_dist_dron = matriz_distancias_dron(nodos)

# 
matriz_costos_camion, matriz_costos_dron = calcular_matrices_costos(
    matriz_dist_camion, parametros['costo_camion'], matriz_dist_dron, parametros['costo_dron']
)

matriz_tiempos_camion, matriz_tiempos_dron = calcular_matrices_tiempos(
    matriz_dist_camion, parametros['velocidad_camion'], matriz_dist_dron, parametros['velocidad_dron']
)
#libreria 
matriz_feromonas_camion = matriz_feromonas(nodos, parametros['feromona_inicial'])

In [7]:
def crear_ruta(lista_nodos,nodo_inicial,parametros,costos,tiempos,feromonas):
    n = parametros['cant_nodos']
    ruta_mixta = [nodo_inicial]
    porVisitar = [i for i in range(parametros['cant_nodos'])]
    porVisitar.remove(nodo_inicial)
    i = nodo_inicial
    
    
    while porVisitar:
        r.seed()
        #ELECCION SIGUIENTE NODO RUTA
        probabilidades = []
        suma_total = 0
        for j in porVisitar:
            p = (feromonas[getF(i,j)]**parametros['alfa'])*((parametros['K']/float(costos[getC(i,j)]))**parametros['beta'])
            suma_total += p
            probabilidades.append([j,p])
        if suma_total == 0:
            # Elige un nodo al azar si suma_total es 0 para evitar la división por cero
            siguiente_nodo = r.choice(porVisitar)
        else:
            siguiente = r.random()
            suma = 0
            for k in probabilidades:
                suma += k[1]
                if siguiente <= suma / suma_total:
                    siguiente_nodo = k[0]
                    break

        porVisitar.remove(siguiente_nodo)
        ruta_mixta.append(siguiente_nodo)
        i = ruta_mixta[-1]

        
    costo_ruta, tiempo_ruta = costo_tiempo(ruta_mixta,costos,tiempos,parametros)#aqui termina aACO
    
    nodos_disponibles = calcular_nodos_disponibles(lista_nodos,parametros,ruta_mixta,tiempos,costos)
    nodos_disponibles = sorted(nodos_disponibles, key=lambda x: x['costo'])
    
     # LA MEJOR DENTRO DE LO POSIBLE (falta aplicar heuristicas)
    bloqueados = set([])
    nodos_dron = []
    costo = 0
    tiempo = 0
    for nodo in nodos_disponibles:
        if not nodo['nodo'] in bloqueados and nodo['costo'] < 0:
            nodos_dron.append(nodo['nodo'])
            bloqueados.update(nodo['bloqueados'])
            costo += nodo['costo']
            tiempo += nodo['tiempo']
    
    if costo_ruta + costo < costo_ruta:
        costo_ruta += costo
        tiempo_ruta += tiempo
        
    ruta_camion = [nodo for nodo in ruta_mixta if nodo not in nodos_dron]
    ruta_dron = ruta_mixta
    
    # Verificar si el nodo inicial ya está en la ruta antes de agregarlo
    if ruta_camion[-1] != nodo_inicial:
        ruta_camion.append(nodo_inicial)

    if ruta_dron[-1] != nodo_inicial:
        ruta_dron.append(nodo_inicial)

    return ruta_camion, ruta_dron, costo_ruta, tiempo_ruta

def calcular_nodos_disponibles(lista_nodos, parametros, ruta_mixta, tiempos, costos):
    disponibles = []
    for nodo in range(len(lista_nodos)):
        if lista_nodos[nodo]['demand'] == 1:  # Si el nodo tiene demanda (es un cliente)
            pos = ruta_mixta.index(nodo)  # Encuentra la posición del nodo en la ruta
            anterior = ruta_mixta[pos-1]  # Nodo anterior en la ruta
            actual = ruta_mixta[pos]  # Nodo actual (cliente)
            siguiente = ruta_mixta[(pos+1)%len(ruta_mixta)]  # Nodo siguiente en la ruta

            # Calcula el tiempo que tomaría para el camión ir directamente del nodo anterior al siguiente
            tiempo_camion = tiempos[getC(anterior, siguiente)]

            # Calcula el tiempo que tomaría para el dron atender el nodo actual desde el anterior y volver al siguiente
            tiempo_dron = tiempos[getD(anterior, actual)] + tiempos[getD(actual, siguiente)] 

            # Añade los tiempos de espera para el camión y el dron
            tiempo_camion += parametros['tiempo_espera_camion'] + parametros['tiempo_servicio_camion']
            tiempo_dron += parametros['tiempo_espera_dron'] + parametros['tiempo_servicio_dron']

            # Compara si es más eficiente enviar el dron
            if tiempo_camion < tiempo_dron and tiempo_dron < parametros['max_vuelo']:
                #cuanto mas tiempo se agrega?
                tiempo_antes = tiempos[getC(anterior,actual)] + tiempos[getC(actual,siguiente)]
                tiempo_nueva = tiempo_dron
                tiempo = tiempo_nueva - tiempo_antes + parametros['tiempo_recuperacion_dron']
                # Calcula el cambio en costo y tiempo
                costo_antes = costos[getC(anterior, actual)] + costos[getC(actual, siguiente)]
                costo_nueva = costos[getC(anterior, siguiente)] + costos[getD(anterior, actual)] + costos[getD(actual, siguiente)]
                costo = costo_nueva - costo_antes
                

                # Si el cambio en costo o tiempo es favorable, considera el nodo como disponible para el dron
                if costo < 0 or tiempo < 0:
                    info = {'nodo': nodo, 'tiempo': tiempo, 'costo': costo, 'bloqueados': [anterior, siguiente]}
                    disponibles.append(info)

    return disponibles


    

In [8]:

def updateMatriz(matriz1,matriz2):
    arreglo = []
    for i in range(len(matriz1)):
        for j in range(len(matriz1)):
            if j >= i:
                arreglo.append(matriz1[i][j])
            else:
                arreglo.append(matriz2[i][j])
    return arreglo
def updateMatrizF(matriz):
    arreglo = []
    for i in range(len(matriz)-1):
        for j in range(i+1,len(matriz)):
            arreglo.append(matriz[i][j])
    return arreglo

#def getF(i,j): return sum(n - x for x in range(min(i, j)))-n + (abs(i-j)-1)
def getF(i,j): return sum(n - x for x in range(1,min(i, j)+1)) + (abs(i-j)-1)

def getC(i,j): return min(i,j)*n+max(i,j)
def getD(i,j): return max(i,j)*n+min(i,j)

def costo_tiempo(ruta_camion, costos, tiempos, parametros):
    n = len(ruta_camion)
    ruta_camion.append(ruta_camion[0])
    costo_total = 0
    tiempo_total = 0
    for i in range(len(ruta_camion)-1):
        anterior = ruta_camion[i]  # Nodo anterior en la ruta
        actual = ruta_camion[i+1]  # Nodo actual en la ruta

        # Añade el costo de viajar del nodo anterior al actual
        costo_total += costos[getC(anterior, actual)]

        # Añade el tiempo de viaje del nodo anterior al actual
        tiempo_total += tiempos[getC(anterior, actual)]

        # Añade los tiempos de espera del camión y del dron
        tiempo_total += parametros['tiempo_espera_camion'] + parametros['tiempo_servicio_camion']

    return costo_total, tiempo_total



def promedio_feromonas(feromonas,parametros,MMAS):
    sumaC = 0
    for i in range(parametros['cant_nodos']):
        for j in range(i):
            sumaC += feromonas[getF(i,j)]-MMAS['min_feromonas']
    promedioC = sumaC/float(parametros['cant_nodos'])
    return promedioC


def ejecutar_traducir(lista_nodos,parametros,costosC,costosD,tiemposC,tiemposD,feromonas):
    global n
    n = parametros['cant_nodos']
    costos = updateMatriz(costosC,costosD)
    tiempos = updateMatriz(tiemposC,tiemposD)
    feromonas = updateMatrizF(feromonas)
    tiempo, mejor_camion, mejor_dron, mejor_costo, mejor_tiempo, ultima_mejora_iteracion = ejecutar(lista_nodos, parametros, costos, tiempos, feromonas)
    return tiempo, mejor_camion, mejor_dron, mejor_costo, mejor_tiempo, ultima_mejora_iteracion
################################################


def ejecutar(lista_nodos,parametros,costos,tiempos,feromonas):
    mejor_camion = []; mejor_dron = []
    mejor_costo = 0; mejor_tiempo = 0
    mejor_registro = None
    ultima_mejora_iteracion = 0  # Guarda la última iteración en la que hubo una mejora en el costo
    
    tiempo_inicial = time.time()
    
    for it in range(parametros['cant_iteraciones']):
        # Aquí calculamos max_feromonas y min_feromonas dinámicamente
        if mejor_costo > 0:
            max_feromonas = 1 / mejor_costo
           
        else:
            max_feromonas = 1  # Un valor predeterminado en caso de mejor_costo = 0

        min_feromonas = 0.1 * max_feromonas  # min_feromonas es el 10% de max_feromonas

        MMAS = {
            'max_feromonas': max_feromonas,
            'min_feromonas': min_feromonas
        }
        
        local_camion = []; local_dron = []
        local_costo = 0; local_tiempo = 0
        local_feromonas = []
        
        for h in range(parametros['cant_hormigas']):
            nodo_inicial = h % parametros['cant_nodos']
            ruta_camion, ruta_dron, costo, tiempo = crear_ruta(lista_nodos,nodo_inicial,parametros,costos,tiempos,feromonas)
            feromonas = feromonas_local(parametros,MMAS,ruta_camion,ruta_dron,costo,tiempo,feromonas)
            
            if local_costo == 0 or costo < local_costo:
                local_camion = list(ruta_camion)
                local_dron = list(ruta_dron)
                local_costo = costo
                local_tiempo = tiempo
                local_feromonas = list(feromonas)
            
            if mejor_costo == 0 or costo < mejor_costo:
                mejor_camion = list(ruta_camion)
                mejor_dron = list(ruta_dron)
                mejor_costo = costo
                mejor_tiempo = tiempo
                mejor_feromonas = list(feromonas)
                ultima_mejora_iteracion = it  # Actualizar la iteración de la última mejora
        
        feromonas = feromonas_global(parametros,MMAS,local_camion,local_dron,local_costo,local_tiempo,feromonas)
    
    return time.time() - tiempo_inicial, mejor_camion, mejor_dron, mejor_costo, mejor_tiempo,ultima_mejora_iteracion

def feromonas_local(parametros,MMAS,ruta_camion,ruta_dron,costo,tiempo,feromonas):
    n = parametros['cant_nodos']
    feromonas_depositadas = parametros['Q']/float(costo)
    for x in range(len(ruta_dron)-1):
        i = ruta_dron[x]
        j = ruta_dron[x+1]
        feromonas[getF(i,j)] += feromonas_depositadas
        if feromonas[getF(i,j)] > MMAS['max_feromonas']:
            feromonas[getF(i,j)] = MMAS['max_feromonas']
    return feromonas

def feromonas_global(parametros,MMAS,ruta_camion,ruta_dron,costo,tiempo,feromonas):
    n = parametros['cant_nodos']
    feromonas_depositadas = parametros['Q']/float(costo)
    for x in range(len(feromonas)):
        feromonas[x] = (1-parametros['evaporacion'])*feromonas[x]
        if feromonas[x] < MMAS['min_feromonas']:
            feromonas[x] = MMAS['min_feromonas']
    for x in range(len(ruta_dron)-1):
        i = ruta_dron[x]
        j = ruta_dron[x+1]
        feromonas[getF(i,j)] += feromonas_depositadas
        if feromonas[getF(i,j)] > MMAS['max_feromonas']:
            feromonas[getF(i,j)] = MMAS['max_feromonas']
    return feromonas
            

In [9]:

tiempo_total, mejor_camion, mejor_dron, mejor_costo, mejor_tiempo, mejor_it = ejecutar_traducir(nodos, parametros, matriz_costos_camion, matriz_costos_dron, matriz_tiempos_camion, matriz_tiempos_dron, matriz_feromonas_camion)
tiempo_fin = time.time()
# Calcular el tiempo total de ejecución
tiempo_total = tiempo_fin - tiempo_inicio
print('Tiempo total de ejecución:', tiempo_total, 'segundos')

Tiempo total de ejecución: 295.57719373703003 segundos


In [10]:
tiempo_total

295.57719373703003

In [11]:
print("Mejor ruta del camion:", mejor_camion)
print("Mejor ruta del dron:", mejor_dron)
print("Mejor costo de la ruta:", mejor_costo)
print("Mejor tiempo de la ruta:", mejor_tiempo)
print("Tiempo CPU:", tiempo_total)
print("Mejor iteracion:", mejor_it)

Mejor ruta del camion: [26, 20, 30, 29, 0, 1, 2, 3, 5, 6, 14, 12, 8, 7, 9, 22, 21, 10, 28, 11, 13, 15, 16, 17, 19, 4, 27, 18, 24, 23, 25, 26]
Mejor ruta del dron: [26, 20, 30, 29, 0, 1, 2, 3, 5, 6, 14, 12, 8, 7, 9, 22, 21, 10, 28, 11, 13, 15, 16, 17, 19, 4, 27, 18, 24, 23, 25, 26]
Mejor costo de la ruta: 158.56182870000004
Mejor tiempo de la ruta: 16165.028007272726
Tiempo CPU: 295.57719373703003
Mejor iteracion: 36


In [12]:
def reorganizar_ruta(ruta):
    # Encuentra el índice del primer cero en la ruta
    indice_cero = ruta.index(0)

    # Reorganiza la ruta para que comience desde el cero
    ruta_reorganizada = ruta[indice_cero:] + ruta[:indice_cero]

    # Elimina duplicados consecutivos (en este caso, el nodo donde se hace el cambio)
    ruta_final = [ruta_reorganizada[i] for i in range(len(ruta_reorganizada)) if i == 0 or ruta_reorganizada[i] != ruta_reorganizada[i-1]]

    return ruta_final

# Reorganizar rutas
mejor_camion_reorganizado = reorganizar_ruta(mejor_camion)
mejor_dron_reorganizado = reorganizar_ruta(mejor_dron)
mejor_camion_reorganizado.append(0)
mejor_dron_reorganizado.append(0)
print("Mejor ruta del camion reorganizada:", mejor_camion_reorganizado)
print("Mejor ruta del dron reorganizada:", mejor_dron_reorganizado)


Mejor ruta del camion reorganizada: [0, 1, 2, 3, 5, 6, 14, 12, 8, 7, 9, 22, 21, 10, 28, 11, 13, 15, 16, 17, 19, 4, 27, 18, 24, 23, 25, 26, 20, 30, 29, 0]
Mejor ruta del dron reorganizada: [0, 1, 2, 3, 5, 6, 14, 12, 8, 7, 9, 22, 21, 10, 28, 11, 13, 15, 16, 17, 19, 4, 27, 18, 24, 23, 25, 26, 20, 30, 29, 0]


In [13]:
import osmnx as ox
import folium
import folium.plugins as plugins

# Configuración inicial
central_lat, central_lon = -33.456574386393314, -70.65970917320084
# Descarga del grafo de OSMnx
G = ox.graph_from_bbox(north, south, east, west, network_type='drive')

# Calcula las rutas para el camión
calculated_route_segments = []
for i in range(len(mejor_camion_reorganizado) - 1):
    origen = ox.distance.nearest_nodes(G, float(nodos[mejor_camion_reorganizado[i]]['long']), float(nodos[mejor_camion_reorganizado[i]]['lat']))
    destino = ox.distance.nearest_nodes(G, float(nodos[mejor_camion_reorganizado[i + 1]]['long']), float(nodos[mejor_camion_reorganizado[i + 1]]['lat']))
    route = ox.shortest_path(G, origen, destino, weight='length')
    route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
    calculated_route_segments.append(route_coords)

# Crea el mapa base
m = folium.Map(location=[central_lat, central_lon], zoom_start=14)

# Dibuja las rutas del camión con flechas
for segment in calculated_route_segments:
    plugins.AntPath(locations=segment, color="blue", weight=5, opacity=0.7, delay=1000).add_to(m)

# Dibuja la ruta del drón con flechas
for i in range(len(mejor_dron_reorganizado) - 1):
    start_coords = (float(nodos[mejor_dron_reorganizado[i]]['lat']), float(nodos[mejor_dron_reorganizado[i]]['long']))
    end_coords = (float(nodos[mejor_dron_reorganizado[i + 1]]['lat']), float(nodos[mejor_dron_reorganizado[i + 1]]['long']))

    # Verifica si el dron está viajando en el camión en esta sección
    if mejor_dron_reorganizado[i] in mejor_camion_reorganizado and mejor_dron_reorganizado[i + 1] in mejor_camion:
        # Encuentra el segmento correspondiente de la ruta del camión
        index_camion_start = mejor_camion_reorganizado.index(mejor_dron_reorganizado[i])
        index_camion_end = mejor_camion_reorganizado.index(mejor_dron_reorganizado[i + 1])

        # Verifica si los índices son consecutivos en la ruta del camión
        if index_camion_end == index_camion_start + 1:
            segmento_camion = calculated_route_segments[index_camion_start]
            plugins.AntPath(locations=segmento_camion, color="purple", weight=5, opacity=0.7, delay=1000, dash_array='5,5').add_to(m)
        else:
            # Dron y camión no están viajando juntos en esta sección
            plugins.AntPath(locations=[start_coords, end_coords], color="red", weight=5, opacity=0.7, delay=1000).add_to(m)
    else:
        # Estilo de línea con flechas para el dron
        plugins.AntPath(locations=[start_coords, end_coords], color="red", weight=5, opacity=0.7, delay=1000).add_to(m)

# Función para añadir marcadores
def add_marker(m, location, popup_text, delivery_type):
    icon_color = "blue" if delivery_type == "truck" else "red"
    icon_type = "truck" if delivery_type == "truck" else "plane"
    folium.Marker(
        location=location,
        popup=popup_text,
        icon=folium.Icon(icon=icon_type, color=icon_color, prefix='fa')
    ).add_to(m)

# Agrega marcadores para los nodos con información de entrega
for i, nodo in enumerate(nodos):
    location = (float(nodo['lat']), float(nodo['long']))
    if i in mejor_camion:
        delivery_type = 'truck'
    elif i in mejor_dron:
        delivery_type = 'drone'
    else:
        continue  # Si el nodo no está en ninguna ruta, no añadir marcador

    popup_text = f"Entrega {i}: {delivery_type.capitalize()}"
    add_marker(m, location, popup_text, delivery_type)

# Alamcane
almacen = (float(nodos[0]['lat']), float(nodos[0]['long']))

# Encuentra la ruta de regreso al almacén para el camión
ultimo_nodo_camion = (float(nodos[mejor_camion_reorganizado[-1]]['lat']), float(nodos[mejor_camion_reorganizado[-1]]['long']))
origen = ox.distance.nearest_nodes(G, ultimo_nodo_camion[1], ultimo_nodo_camion[0])
destino = ox.distance.nearest_nodes(G, almacen[1], almacen[0])
ruta_regreso = ox.shortest_path(G, origen, destino, weight='length')
ruta_regreso_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in ruta_regreso]

# Dibuja la ruta de regreso con flechas
if mejor_dron_reorganizado[-1] == mejor_camion_reorganizado[-1]:
    # Dron y camión regresan juntos
    for i in range(len(ruta_regreso_coords) - 1):
        plugins.AntPath(locations=[ruta_regreso_coords[i], ruta_regreso_coords[i + 1]], color="blue", weight=5, opacity=0.7, delay=1000).add_to(m)
        plugins.AntPath(locations=[ruta_regreso_coords[i], ruta_regreso_coords[i + 1]], color="purple", weight=5, opacity=0.7, delay=1000, dash_array='5,5').add_to(m)
else:
    # Solo el camión regresa
    plugins.AntPath(locations=ruta_regreso_coords, color="blue", weight=5, opacity=0.7, delay=1000).add_to(m)

# Agrega un marcador para el almacén
folium.Marker(
    location=almacen,
    icon=folium.Icon(color="green", icon="home")
).add_to(m)

# Mostrar el mapa
m


In [14]:
import osmnx as ox
import folium
import folium.plugins as plugins

# Configuración inicial
central_lat, central_lon = -33.456574386393314, -70.65970917320084
# Descarga del grafo de OSMnx
G = ox.graph_from_bbox(north, south, east, west, network_type='drive')

# Calcula las rutas para el camión
calculated_route_segments = []
for i in range(len(mejor_camion) - 1):
    origen = ox.distance.nearest_nodes(G, float(nodos[mejor_camion[i]]['long']), float(nodos[mejor_camion[i]]['lat']))
    destino = ox.distance.nearest_nodes(G, float(nodos[mejor_camion[i + 1]]['long']), float(nodos[mejor_camion[i + 1]]['lat']))
    route = ox.shortest_path(G, origen, destino, weight='length')
    route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
    calculated_route_segments.append(route_coords)

# Crea el mapa base
m = folium.Map(location=[central_lat, central_lon], zoom_start=14)

# Dibuja las rutas del camión con flechas
for segment in calculated_route_segments:
    plugins.AntPath(locations=segment, color="blue", weight=5, opacity=0.7, delay=1000).add_to(m)

# Dibuja la ruta del drón con flechas
for i in range(len(mejor_dron) - 1):
    start_coords = (float(nodos[mejor_dron[i]]['lat']), float(nodos[mejor_dron[i]]['long']))
    end_coords = (float(nodos[mejor_dron[i + 1]]['lat']), float(nodos[mejor_dron[i + 1]]['long']))

    # Verifica si el dron está viajando en el camión en esta sección
    if mejor_dron[i] in mejor_camion and mejor_dron[i + 1] in mejor_camion:
        # Encuentra el segmento correspondiente de la ruta del camión
        index_camion_start = mejor_camion.index(mejor_dron[i])
        index_camion_end = mejor_camion.index(mejor_dron[i + 1])

        # Verifica si los índices son consecutivos en la ruta del camión
        if index_camion_end == index_camion_start + 1:
            segmento_camion = calculated_route_segments[index_camion_start]
            plugins.AntPath(locations=segmento_camion, color="purple", weight=5, opacity=0.7, delay=1000, dash_array='5,5').add_to(m)
        else:
            # Dron y camión no están viajando juntos en esta sección
            plugins.AntPath(locations=[start_coords, end_coords], color="red", weight=5, opacity=0.7, delay=1000).add_to(m)
    else:
        # Estilo de línea con flechas para el dron
        plugins.AntPath(locations=[start_coords, end_coords], color="red", weight=5, opacity=0.7, delay=1000).add_to(m)

# Función para añadir marcadores
def add_marker(m, location, popup_text, delivery_type):
    icon_color = "blue" if delivery_type == "truck" else "red"
    icon_type = "truck" if delivery_type == "truck" else "plane"
    folium.Marker(
        location=location,
        popup=popup_text,
        icon=folium.Icon(icon=icon_type, color=icon_color, prefix='fa')
    ).add_to(m)

# Agrega marcadores para los nodos con información de entrega
for i, nodo in enumerate(nodos):
    location = (float(nodo['lat']), float(nodo['long']))
    if i in mejor_camion or i in mejor_dron:
        delivery_type = 'truck' if i in mejor_camion else 'drone'
        popup_text = f"Entrega {i}: {delivery_type.capitalize()}"
        add_marker(m, location, popup_text, delivery_type)

# Almacén
almacen = (float(nodos[0]['lat']), float(nodos[0]['long']))
folium.Marker(
    location=almacen,
    popup="Almacén",
    icon=folium.Icon(color="green", icon="home")
).add_to(m)

# Mostrar el mapa
m

In [15]:
Mejor ruta del camion: [2, 10, 6, 8, 4, 5, 0, 1, 2]
Mejor ruta del dron: [2, 9, 10, 6, 3, 8, 4, 7, 5, 0, 1, 2]
Mejor costo de la ruta: 37.016880778685454
Mejor tiempo de la ruta: 5577.712599950018


SyntaxError: invalid syntax (1516791291.py, line 1)

In [ ]:
import osmnx as ox
import folium
import folium.plugins as plugins

# Configuración inicial
central_lat, central_lon = -33.456574386393314, -70.65970917320084
# Descarga del grafo de OSMnx
G = ox.graph_from_bbox(north, south, east, west, network_type='drive')

# Calcula las rutas para el camión
calculated_route_segments = []
for i in range(len(mejor_camion_reorganizado) - 1):
    origen = ox.distance.nearest_nodes(G, float(nodos[mejor_camion_reorganizado[i]]['long']), float(nodos[mejor_camion_reorganizado[i]]['lat']))
    destino = ox.distance.nearest_nodes(G, float(nodos[mejor_camion_reorganizado[i + 1]]['long']), float(nodos[mejor_camion_reorganizado[i + 1]]['lat']))
    route = ox.shortest_path(G, origen, destino, weight='length')
    route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
    calculated_route_segments.append(route_coords)

# Crea el mapa base
m = folium.Map(location=[central_lat, central_lon], zoom_start=14)

# Dibuja las rutas del camión con flechas
for segment in calculated_route_segments:
    plugins.AntPath(locations=segment, color="blue", weight=5, opacity=0.7, delay=1000).add_to(m)

# # Dibuja la ruta del drón con flechas
# for i in range(len(mejor_dron_reorganizado) - 1):
#     start_coords = (float(nodos[mejor_dron_reorganizado[i]]['lat']), float(nodos[mejor_dron_reorganizado[i]]['long']))
#     end_coords = (float(nodos[mejor_dron_reorganizado[i + 1]]['lat']), float(nodos[mejor_dron_reorganizado[i + 1]]['long']))

#     # Verifica si el dron está viajando en el camión en esta sección
#     if mejor_dron_reorganizado[i] in mejor_camion_reorganizado and mejor_dron_reorganizado[i + 1] in mejor_camion:
#         # Encuentra el segmento correspondiente de la ruta del camión
#         index_camion_start = mejor_camion_reorganizado.index(mejor_dron_reorganizado[i])
#         index_camion_end = mejor_camion_reorganizado.index(mejor_dron_reorganizado[i + 1])

#         # Verifica si los índices son consecutivos en la ruta del camión
#         if index_camion_end == index_camion_start + 1:
#             segmento_camion = calculated_route_segments[index_camion_start]
#             plugins.AntPath(locations=segmento_camion, color="purple", weight=5, opacity=0.7, delay=1000, dash_array='5,5').add_to(m)
#         else:
#             # Dron y camión no están viajando juntos en esta sección
#             plugins.AntPath(locations=[start_coords, end_coords], color="red", weight=5, opacity=0.7, delay=1000).add_to(m)
#     else:
#         # Estilo de línea con flechas para el dron
#         plugins.AntPath(locations=[start_coords, end_coords], color="red", weight=5, opacity=0.7, delay=1000).add_to(m)

# Función para añadir marcadores
def add_marker(m, location, popup_text, delivery_type):
    icon_color = "blue" if delivery_type == "truck" else "red"
    icon_type = "truck" if delivery_type == "truck" else "plane"
    folium.Marker(
        location=location,
        popup=popup_text,
        icon=folium.Icon(icon=icon_type, color=icon_color, prefix='fa')
    ).add_to(m)

# Agrega marcadores para los nodos con información de entrega
for i, nodo in enumerate(nodos):
    location = (float(nodo['lat']), float(nodo['long']))
    if i in mejor_camion:
        delivery_type = 'truck'
    elif i in mejor_dron:
        delivery_type = 'drone'
    else:
        continue  # Si el nodo no está en ninguna ruta, no añadir marcador

    popup_text = f"Entrega {i}: {delivery_type.capitalize()}"
    add_marker(m, location, popup_text, delivery_type)

# Alamcane
almacen = (float(nodos[0]['lat']), float(nodos[0]['long']))

# Encuentra la ruta de regreso al almacén para el camión
ultimo_nodo_camion = (float(nodos[mejor_camion_reorganizado[-1]]['lat']), float(nodos[mejor_camion_reorganizado[-1]]['long']))
origen = ox.distance.nearest_nodes(G, ultimo_nodo_camion[1], ultimo_nodo_camion[0])
destino = ox.distance.nearest_nodes(G, almacen[1], almacen[0])
ruta_regreso = ox.shortest_path(G, origen, destino, weight='length')
ruta_regreso_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in ruta_regreso]

# Dibuja la ruta de regreso con flechas
if mejor_dron_reorganizado[-1] == mejor_camion_reorganizado[-1]:
    # Dron y camión regresan juntos
    for i in range(len(ruta_regreso_coords) - 1):
        plugins.AntPath(locations=[ruta_regreso_coords[i], ruta_regreso_coords[i + 1]], color="blue", weight=5, opacity=0.7, delay=1000).add_to(m)
        plugins.AntPath(locations=[ruta_regreso_coords[i], ruta_regreso_coords[i + 1]], color="purple", weight=5, opacity=0.7, delay=1000, dash_array='5,5').add_to(m)
else:
    # Solo el camión regresa
    plugins.AntPath(locations=ruta_regreso_coords, color="blue", weight=5, opacity=0.7, delay=1000).add_to(m)

# Agrega un marcador para el almacén
folium.Marker(
    location=almacen,
    icon=folium.Icon(color="green", icon="home")
).add_to(m)

# Mostrar el mapa
m
